In [ ]:
pip install pyspark

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import functions as sf
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import IntegerType

In [ ]:
#setup configuration property 
#set the master URL 
#set an application name 
conf = SparkConf().setMaster("local").setAppName("sparkproject")
#start spark cluster 
#if already started then get it else start it 
sc = SparkContext.getOrCreate(conf=conf)
#initialize SQLContext from spark cluster 
sqlContext = SQLContext(sc)

In [ ]:
#Filepath variable for your file location directory
FilePath="/content/"
#FileName variable
FileName="flights_small.csv"
#combine both above variables
FullPath= FilePath + FileName

In [ ]:
#dataframe 
#set header property true for the actual header columns
df=sqlContext.read.csv(FullPath, header=True)
#display data from the dataframe
df.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
flights_small = flights_small.withColumn('Full_Date', 
                    sf.concat(sf.col('day'),sf.lit('/'), sf.col('month'),sf.lit('/'), sf.col('year')))
flights_small.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute| Full_Date|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58| 8/12/2014|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40| 22/1/2014|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|  9/3/2014|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|  9/4/2014|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA

In [ ]:
#Filepath variable for your file location directory
FilePath2="/content/"
#FileName variable
FileName2="homelessness.csv"
#combine both above variables
FullPath2= FilePath2 + FileName2

In [ ]:
#dataframe 
#set header property true for the actual header columns
homelessness=sqlContext.read.csv(FullPath2, header=True)
#convert the Data type of two Column
homelessness = homelessness.withColumn("individuals", homelessness["individuals"].cast(IntegerType()))
homelessness = homelessness.withColumn("family_members", homelessness["family_members"].cast(IntegerType()))
#display data from the dataframe
homelessness.show()

+---+------------------+--------------------+-----------+--------------+---------+
|_c0|            region|               state|individuals|family_members|state_pop|
+---+------------------+--------------------+-----------+--------------+---------+
|  0|East South Central|             Alabama|       2570|           864|  4887681|
|  1|           Pacific|              Alaska|       1434|           582|   735139|
|  2|          Mountain|             Arizona|       7259|          2606|  7158024|
|  3|West South Central|            Arkansas|       2280|           432|  3009733|
|  4|           Pacific|          California|     109008|         20964| 39461588|
|  5|          Mountain|            Colorado|       7607|          3250|  5691287|
|  6|       New England|         Connecticut|       2280|          1696|  3571520|
|  7|    South Atlantic|            Delaware|        708|           374|   965479|
|  8|    South Atlantic|District of Columbia|       3770|          3134|   701547|
|  9

In [ ]:
#creating the low cardinality from AVG values of a column
homelessness = homelessness.withColumn("LABLE_individuals", 
                                       sf.when(homelessness.individuals 
                                               < str(homelessness.select(sf.avg("individuals")).collect()[0][0]) 
                                               , "LOW").otherwise("HIGH"))
homelessness = homelessness.withColumn("LABLE_family_members", 
                                       sf.when(homelessness.family_members 
                                               < str(homelessness.select(sf.avg("family_members")).collect()[0][0]) 
                                               , "LOW").otherwise("HIGH"))
homelessness.show()

+---+------------------+--------------------+-----------+--------------+---------+-----------------+--------------------+
|_c0|            region|               state|individuals|family_members|state_pop|LABLE_individuals|LABLE_family_members|
+---+------------------+--------------------+-----------+--------------+---------+-----------------+--------------------+
|  0|East South Central|             Alabama|       2570|           864|  4887681|              LOW|                 LOW|
|  1|           Pacific|              Alaska|       1434|           582|   735139|              LOW|                 LOW|
|  2|          Mountain|             Arizona|       7259|          2606|  7158024|             HIGH|                 LOW|
|  3|West South Central|            Arkansas|       2280|           432|  3009733|              LOW|                 LOW|
|  4|           Pacific|          California|     109008|         20964| 39461588|             HIGH|                HIGH|
|  5|          Mountain|